<a href="https://colab.research.google.com/github/rfagaraz/Excel_REGEX_Concatenator/blob/master/Filtrador_de_Leads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import glob
import re
from openpyxl import load_workbook


In [ ]:
leads = pd.DataFrame()
for files in glob.glob('*.xlsx'):
  df = pd.read_excel(files)
  leads = leads.append(df, ignore_index=False)


In [ ]:
book = load_workbook('/content/Template file/Template.xlsx')
writer = pd.ExcelWriter('/content/Template file/Template.xlsx', engine = 'openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

In [ ]:
leads.replace(to_replace ="^(\+55|55|055|\s\+55|\s55|\s055){0,1}?(\s|-|\.|\+|\_)?(0{0,1}\s{0,1})?([1-9][0-9]|\([1-9][0-9]\))(\s|-|\.|\+|\_|\(|\)){0,2}?((?!9999)\d{4,5}){0,1}?(\s|-|\.|\+|\_)?((?!0000)?\d{4,5})$", value=r"\4-\6\8", regex=True, inplace=True)
leads.filter(regex="^(\+55|55|055|\s\+55|\s55|\s055){0,1}?(\s|-|\.|\+|\_)?(0{0,1}\s{0,1})?([1-9][0-9]|\([1-9][0-9]\))(\s|-|\.|\+|\_){0,2}?(\d{4,5}){0,1}?(\s|-|\.|\+|\_)?((?!0000)?\d{4,5})$", axis=1)
leads.insert(6, column = 'validação de número telefônico', value = 'Sem erros identificados')


In [ ]:
for index, row in leads.iterrows():
  fone = leads['Phone Mask'][index]
  try:
      if len(fone) == 12 and fone[3] != '9':
         leads['validação de número telefônico'][index] = 'Não começa com 9'
      elif fone[1] == '0':
         leads['validação de número telefônico'][index] = 'DDD inválido'
      elif fone[0] == '0':
         leads['validação de número telefônico'][index] = 'DDD inválido'
      elif len(fone) < 11:
         leads['validação de número telefônico'][index] = 'Curto demais'
      elif len(fone) > 12:
         leads['validação de número telefônico'][index] = 'Longo demais'
      elif len(fone) == 11 and (fone[3] == '6' or fone[3] == '7' or fone[3] == '8' or fone[3] == '9'):
         leads['validação de número telefônico'][index] = 'Não tem o nono dígito'
      elif len(fone) > 12:
         leads['validação de número telefônico'][index] = 'Longo demais'
      elif len(fone) == 'NaN':
         leads['validação de número telefônico'][index] = 'Telefone não informado'
            
  except:
      leads['validação de número telefônico'][index] = 'Não conseguiu identificar o número'

In [ ]:
leads.to_excel(writer, sheet_name='Base Original')
writer.save()